# Workbook for Solving a TSP

In [1]:
import veroviz as vrv

In [2]:
vrv.checkVersion()

'Your current installed version of veroviz is 0.4.0, the latest version available is 0.4.3. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

In [3]:
ORS_API_KEY='5b3ce3597851110001cf62480fb8f5604a0b485b911812eedd4d31cf'

# 1 -- Create some "nodes"

In [4]:
myBoundingRegion = [[43.01770145196991, -78.87840270996095], [42.878480017739044, -78.8756561279297], [42.83569550641454, -78.68270874023439], [42.96596996868038, -78.60717773437501], [43.04430359661548, -78.72528076171876]]
myBoundingRegion

[[43.01770145196991, -78.87840270996095],
 [42.878480017739044, -78.8756561279297],
 [42.83569550641454, -78.68270874023439],
 [42.96596996868038, -78.60717773437501],
 [43.04430359661548, -78.72528076171876]]

In [5]:
nodesDF = vrv.generateNodes(nodeType = 'customer',
                            nodeName = 'cust',
                            numNodes = 10,
                            incrementName = True,
                            nodeDistrib = 'uniformBB',
                            nodeDistribArgs = {'boundingRegion': myBoundingRegion},
                            snapToRoad = True,
                            dataProvider     = 'ORS-online',
                            dataProviderArgs = {'APIkey' : ORS_API_KEY})
nodesDF

,id,lat,lon,altMeters,nodeName,nodeType,popupText,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText,elevMeters
0,1,42.963622,-78.746143,0,cust1,customer,1,glyphicon,info-sign,blue,1,pin,blue,1,None
1,2,43.008142,-78.851057,0,cust2,customer,2,glyphicon,info-sign,blue,2,pin,blue,2,None
2,3,42.894528,-78.862845,0,cust3,customer,3,glyphicon,info-sign,blue,3,pin,blue,3,None
3,4,42.908515,-78.778640,0,cust4,customer,4,glyphicon,info-sign,blue,4,pin,blue,4,None
4,5,43.011501,-78.792424,0,cust5,customer,5,glyphicon,info-sign,blue,5,pin,blue,5,None
5,6,42.960808,-78.620317,0,cust6,customer,6,glyphicon,info-sign,blue,6,pin,blue,6,None
6,7,42.895875,-78.695463,0,cust7,customer,7,glyphicon,info-sign,blue,7,pin,blue,7,None
7,8,42.877735,-78.660767,0,cust8,customer,8,glyphicon,info-sign,blue,8,pin,blue,8,None
8,9,42.978591,-78.762551,0,cust9,customer,9,glyphicon,info-sign,blue,9,pin,blue,9,None
9,10,42.976594,-78.662126,0,cust10,customer,10,glyphicon,info-sign,blue,10,pin,blue,10,None


In [6]:
len(nodesDF)

10

In [7]:
# View our nodes and bounding region:
vrv.createLeaflet(nodes = nodesDF, boundingRegion=myBoundingRegion)

In [8]:
# 2. Get time and distance matrices
[timeSec, distMeters] = vrv.getTimeDist2D(nodes = nodesDF,
                                          routeType = 'euclidean2D',
                                          speedMPS = vrv.convertSpeed(25, 'miles', 'hr', 'meters', 'second'))

In [9]:
#timeSec
timeSec[1,2]

884.3328078191814

In [10]:
# Optional, just to view the data in a table:
vrv.convertMatricesDictionaryToDataframe(timeSec)

,1,2,3,4,5,6,7,8,9,10
1,0.000000,884.332808,1094.701674,596.984964,583.590816,919.026623,768.460137,1057.296334,191.010919,626.701954
2,884.332808,0.000000,1132.625190,1122.617709,429.025126,1748.423385,1592.518817,1900.387622,709.483776,1413.846107
3,1094.701674,1132.625190,0.000000,630.859164,1271.358993,1890.122659,1223.381689,1486.501807,1111.215494,1677.674213
4,596.984964,1122.617709,630.859164,0.000000,1028.641080,1267.821015,620.678989,914.218267,706.410633,1087.180603
5,583.590816,429.025126,1271.358993,1028.641080,0.000000,1353.303894,1349.897549,1640.847246,393.103411,1012.097320
6,919.026623,1748.423385,1890.122659,1267.821015,1353.303894,0.000000,847.291280,877.045239,1053.202755,343.173895
7,768.460137,1592.518817,1223.381689,620.678989,1349.897549,847.291280,0.000000,311.176596,957.139111,838.493059
8,1057.296334,1900.387622,1486.501807,914.218267,1640.847246,877.045239,311.176596,0.000000,1248.134725,982.727649
9,191.010919,709.483776,1111.215494,706.410633,393.103411,1053.202755,957.139111,1248.134725,0.000000,733.245362
10,626.701954,1413.846107,1677.674213,1087.180603,1012.097320,343.173895,838.493059,982.727649,733.245362,0.000000


In [11]:
def tsp_cost(route, costDict):
    cost = 0
    
    i = route[0]
    for j in route[1:]:
        cost += costDict[i,j]
        i = j
        
    cost += costDict[i, route[0]]
    
    return cost

In [12]:
def tsp_neighbor(route):
    # You should document this function to explain what's happening
    #import random
    from random import randint 
    
    #a = random.randint(0,len(route)-3)
    a = randint(0,len(route)-3)
    #b = random.randint(a+1, len(route)-2)
    b = randint(a+1, len(route)-2)
    
    newRoute = []
    newRoute.extend(route[0:a])
    
    subtour = route[a:b+1]
    subtour.reverse()
    newRoute.extend(subtour)
    
    newRoute.extend(route[b+1:len(route)-1])
    
    newRoute.append(newRoute[0])
    
    return newRoute

In [13]:
a=[1,5,8,9,2,4,3,7,6]
tsp_neighbor(a)

[1, 5, 8, 7, 3, 4, 2, 9, 1]

In [14]:
def solve_tsp_nn(startNode, costDict, nodesDF): 
    """
    This function computes a "nearest neighbor" solution to a TSP.
    
    Inputs
    ------
    startNode: Integer, indicating the node where the salesperson begins (and ends) the route
    
    costDict: VeRoViz time or distance dictionary.
    
    nodesDF: VeRoViz nodes dataframe
    
    Returns
    -------
    An ordered list of nodeIDs specifying a TSP route.
    """
    
    # Solve the TSP with a "nearest neighbor" heuristic
    nn_route = []

    # Start our route by visiting the startNode
    nn_route.append(startNode)

    # Initialize a list of unvisited nodes
    unvisitedNodes = list(nodesDF[nodesDF['id'] != startNode]['id'])

    # Let i represent our "current" location:
    i = startNode

    while len(unvisitedNodes) > 0:
        # Initialize minTime to a huge value
        minTime = float('inf')

        # Find the nearest unvisited node to our current node:
        for j in unvisitedNodes:
            if (costDict[i,j] < minTime):
                nextNode = j
                minTime = costDict[i,j]

        # Update our salesperson's location
        i = nextNode

        # Append nextNode to our route:
        nn_route.append(nextNode)

        # Remove nextNode from our list of unvisitedNodes:
        unvisitedNodes.remove(nextNode)

    nn_route.append(startNode)

    return nn_route    

In [15]:
#Using recursion
def solveTSP_SA(nodesDF, costDict, timeLimit):
    import time
    from random import random
    import math as m
    
    runTime=0
    start_time = time.time()
    startnode=1
    T0=4000
    I=3
    d=5
    Tfinal=2020
    cutoffTime=timeLimit
    
    # Initialize by calling your nearest neighbor function
    X0 = solve_tsp_nn(startnode,costDict,nodesDF)
    Z=tsp_cost(X0,costDict)
    print('Intial Solution : X0 =',X0,'Z0 =',Z)
    
    # Simulated annealing (SA) procedure
    Xcur = tsp_neighbor(X0)
    Zcur = tsp_cost(Xcur,costDict)    
    Tcur = T0
    print('Current Solution : Xcur =',Xcur,'Zcur =',Zcur)
    Xbest = Xcur
    Zbest = Zcur
    print('Best known Solution : Xbest =',Xbest,'Zbest =',Zbest)
    
    def solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime):
        print('Tcur=',Tcur)
        for i in range(1,I+1):
            #Generate a neighbor solution, Xcount
            # call tsp_neighbor(route)
            # ...
            # ...
            Xcount=tsp_neighbor(X0)
            Z = tsp_cost(Xcount,costDict)
            print('i=',i)
            print('Xcount=',Xcount,'Zcount=',Z)

            if (Z<Zcur):
                print('Z-Zcur=',Z-Zcur)
                Xcur = Xcount
                Zcur = Z
                print('Xcur=',Xcur,'Zcur=',Z)


            else:
                C=Z-Zcur
                print('C=',C)
                if(random()<=m.exp(-C/Tcur)):
                    Xcur = Xcount
                    Zcur = Z
                    print('Xcur=',Xcur,'Zcur=',Zcur)


            if(Z < Zbest):
                print('Z-Zbest=',Z-Zbest)
                Zbest = Z
                Xbest = Xcur
                print('Xbest=',Xbest,'Zbest=',Zbest)

        Tcur=Tcur-d
        #print('Tcur=',Tcur)
        print('Xbest=',Xbest,'Zbest=',Zbest)
            
        runTime=time.time()-start_time
        print(runTime)
        if(Tcur<Tfinal or runTime>=timeLimit):
            if(Tcur<Tfinal):
                print('Tcur=',Tcur,'< Tfinal=',Tfinal)
                print('runTime =',runTime,',cutoffTime =',cutoffTime)
            elif(runTime>=cutoffTime):
                print('runTime =',runTime,'> cutoffTime =',cutoffTime)
                print('Tcur=',Tcur,',Tfinal=',Tfinal)
            return Xbest

        return solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    
    
    XBEST=solveTSP_SA2(I,X0,Z,Xcur,Zcur,Xbest,Zbest,Tcur,d,Tfinal,runTime)
    
    totalruntime=time.time()-start_time
    print('totaltime=',totalruntime)
    
    # SA produces a "best" solution, X_best.
    # X_best is just a sequence of node numbers,
    # starting from and returning to a "home" location.
    # For example, Xbest = [1, 5, 3, 4, 2, 1].
    
    # Your function should return a VeRoViz "assignments" dataframe.
    # Fortunately, VeRoViz provides a function to make this very easy:
    assignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
        nodeSeq          = XBEST,
        nodes            = nodesDF,
        objectID         = 'Blue Car',
        modelFile        = 'veroviz/models/car_blue.gltf',
        modelScale       = 100,
        modelMinPxSize   = 75,
        routeType        = 'fastest',
        speedMPS         = None,
        leafletColor     = 'blue',
        leafletWeight    = 3,
        leafletStyle     = 'dashed',
        leafletOpacity   = 0.8,
        useArrows        = True,
        cesiumColor      = 'blue',
        cesiumWeight     = 3,
        cesiumStyle      = 'solid',
        cesiumOpacity    = 0.8,
        dataProvider     = 'ORS-online',
        dataProviderArgs = {'APIkey' : ORS_API_KEY})

    # See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromNodeSeq2D
    # for more info on this function.
    
    return assignmentsDF
    #return Xbest

In [16]:
#%time
X_best=solveTSP_SA(nodesDF,timeSec,3)
X_best

Intial Solution : X0 = [1, 9, 5, 2, 4, 7, 8, 6, 10, 3, 1] Z0 = 7060.207771879965
Current Solution : Xcur = [1, 5, 9, 2, 4, 7, 8, 6, 10, 3, 1] Zcur = 7733.246318569067
Best known Solution : Xbest = [1, 5, 9, 2, 4, 7, 8, 6, 10, 3, 1] Zbest = 7733.246318569067
Tcur= 4000
i= 1
Xcount= [1, 9, 5, 7, 4, 2, 8, 6, 10, 3, 1] Zcount= 9570.291220365783
C= 1837.0449017967167
Xcur= [1, 9, 5, 7, 4, 2, 8, 6, 10, 3, 1] Zcur= 9570.291220365783
i= 2
Xcount= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcount= 7493.989289039374
Z-Zcur= -2076.3019313264094
Xcur= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcur= 7493.989289039374
Z-Zbest= -239.25702952969277
Xbest= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zbest= 7493.989289039374
i= 3
Xcount= [1, 9, 4, 2, 5, 7, 8, 6, 10, 3, 1] Zcount= 8102.733553199633
C= 608.7442641602593
Xcur= [1, 9, 4, 2, 5, 7, 8, 6, 10, 3, 1] Zcur= 8102.733553199633
Xbest= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zbest= 7493.989289039374
0.008379697799682617
Tcur= 3995
i= 1
Xcount= [1, 7, 4, 2, 5, 9, 8, 6, 10, 3, 1] 

Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
0.3724651336669922
Tcur= 3900
i= 1
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
Z-Zcur= -814.7573056577585
Xcur= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcur= 8139.156816567327
i= 2
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
C= 0.0
Xcur= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcur= 8139.156816567327
i= 3
Xcount= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcount= 7493.989289039374
Z-Zcur= -645.1675275279531
Xcur= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcur= 7493.989289039374
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
0.37521982192993164
Tcur= 3895
i= 1
Xcount= [1, 9, 7, 4, 2, 5, 8, 6, 10, 3, 1] Zcount= 8953.914122225086
C= 1459.9248331857116
i= 2
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
C= 645.1675275279531
Xcur= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcur= 8139.156816567327
i= 3
Xcount= [1, 9, 5, 2, 4, 7, 8, 10, 6, 3, 1] Zcount= 7378.3386274304

Xcount= [1, 9, 2, 5, 4, 7, 8, 6, 10, 3, 1] Zcount= 7282.611508472787
Z-Zcur= -850.5390430990237
Xcur= [1, 9, 2, 5, 4, 7, 8, 6, 10, 3, 1] Zcur= 7282.611508472787
i= 2
Xcount= [1, 9, 5, 3, 10, 6, 8, 7, 4, 2, 1] Zcount= 7692.172772419486
C= 409.561263946699
Xcur= [1, 9, 5, 3, 10, 6, 8, 7, 4, 2, 1] Zcur= 7692.172772419486
i= 3
Xcount= [1, 9, 5, 2, 6, 8, 7, 4, 10, 3, 1] Zcount= 8430.020156639608
C= 737.847384220122
Xcur= [1, 9, 5, 2, 6, 8, 7, 4, 10, 3, 1] Zcur= 8430.020156639608
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
0.6345357894897461
Tcur= 3775
i= 1
Xcount= [10, 6, 8, 7, 4, 2, 5, 9, 1, 3, 10] Zcount= 7060.207771879965
Z-Zcur= -1369.8123847596435
Xcur= [10, 6, 8, 7, 4, 2, 5, 9, 1, 3, 10] Zcur= 7060.207771879965
i= 2
Xcount= [1, 9, 5, 6, 8, 7, 4, 2, 10, 3, 1] Zcount= 9055.158751635963
C= 1994.9509797559986
Xcur= [1, 9, 5, 6, 8, 7, 4, 2, 10, 3, 1] Zcur= 9055.158751635963
i= 3
Xcount= [1, 9, 5, 2, 8, 7, 4, 6, 10, 3, 1] Zcount= 8228.753461043898
Z-Zcur= -826.4052905

i= 1
Xcount= [1, 9, 2, 5, 4, 7, 8, 6, 10, 3, 1] Zcount= 7282.611508472787
Z-Zcur= -850.5390430990237
Xcur= [1, 9, 2, 5, 4, 7, 8, 6, 10, 3, 1] Zcur= 7282.611508472787
i= 2
Xcount= [1, 9, 3, 10, 6, 8, 7, 4, 2, 5, 1] Zcount= 7267.208997806009
Z-Zcur= -15.402510666777744
Xcur= [1, 9, 3, 10, 6, 8, 7, 4, 2, 5, 1] Zcur= 7267.208997806009
i= 3
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
C= 871.9478187613176
Xcur= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcur= 8139.156816567327
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.5201561450958252
Tcur= 3655
i= 1
Xcount= [1, 9, 5, 10, 6, 8, 7, 4, 2, 3, 1] Zcount= 7098.230941787261
Z-Zcur= -1040.925874780066
Xcur= [1, 9, 5, 10, 6, 8, 7, 4, 2, 3, 1] Zcur= 7098.230941787261
i= 2
Xcount= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcount= 7493.989289039374
C= 395.7583472521128
Xcur= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcur= 7493.989289039374
i= 3
Xcount= [1, 9, 5, 8, 7, 4, 2, 6, 10, 3, 1] Zcount= 9143.408037446596
C= 1649.41

Xcur= [8, 7, 4, 2, 5, 9, 1, 6, 10, 3, 8] Zcur= 7493.989289039374
i= 2
Xcount= [1, 9, 5, 2, 7, 4, 8, 6, 10, 3, 1] Zcount= 8133.150551571811
C= 639.1612625324369
i= 3
Xcount= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcount= 7493.989289039374
C= 0.0
Xcur= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcur= 7493.989289039374
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
2.6661548614501953
Tcur= 3535
i= 1
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
C= 645.1675275279531
Xcur= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcur= 8139.156816567327
i= 2
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
C= 0.0
Xcur= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcur= 8139.156816567327
i= 3
Xcount= [1, 9, 5, 2, 4, 7, 6, 8, 10, 3, 1] Zcount= 8235.876210714734
C= 96.719394147407
Xcur= [1, 9, 5, 2, 4, 7, 6, 8, 10, 3, 1] Zcur= 8235.876210714734
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
2.6671228408813477
Tcur= 3530
i= 1
Xcount= [1, 9, 5, 2, 4, 7, 8, 3

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,ganttColor,popupText,startElevMeters,endElevMeters,wayname,waycategory,surface,waytype,steepness,tollway
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.963622,-78.746143,0,18.400000,...,darkgray,None,204.0,204.0,Rock Street,No category,Asphalt,Street,0,False
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,18.400000,42.964298,-78.746319,0,22.290428,...,darkgray,None,204.0,204.0,Glen Avenue,No category,Asphalt,Street,0,False
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,22.290428,42.964244,-78.746504,0,33.832206,...,darkgray,None,204.0,204.0,Glen Avenue,No category,Asphalt,Street,0,False
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,33.832206,42.964136,-78.747076,0,37.300000,...,darkgray,None,204.0,204.0,Glen Avenue,No category,Asphalt,Street,0,False
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,37.300000,42.964107,-78.747249,0,48.738125,...,darkgray,None,204.0,204.0,North Cayuga Road,No category,Asphalt,Street,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,7047.950922,42.962559,-78.746703,0,7052.430128,...,darkgray,None,204.6,204.3,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1779,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,7052.430128,42.962669,-78.746010,0,7053.400000,...,darkgray,None,204.3,204.2,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1780,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,7053.400000,42.962693,-78.745860,0,7056.795360,...,darkgray,None,204.2,204.1,Rock Street,No category,Asphalt,Street,0,False
1781,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,7056.795360,42.962817,-78.745899,0,7075.242069,...,darkgray,None,204.1,203.6,Rock Street,No category,Asphalt,Street,0,False


In [17]:
vrv.createLeaflet(arcs=X_best, nodes=nodesDF)

In [18]:
# Using loop
def solveTSP_SA_loop(nodesDF, costDict, timeLimit):
    import time
    from random import random
    import math as m
    
    runTime=0
    start_time = time.time()
    startnode=1
    
    
    # Simulated annealing (SA) procedure--Phase I
    # Initialize parameters 
    T0=4000
    I=1000
    d=5
    Tfinal=2020
    cutoffTime=timeLimit
    
    # Initial solution by calling nearest neighbor function
    X0 = solve_tsp_nn(startnode,costDict,nodesDF)
    Z=tsp_cost(X0,costDict)
    print('Intial Solution : X0 =',X0,'Z0 =',Z)
    
    # Find current solution by calling neighbor/subtour function
    Xcur = tsp_neighbor(X0)
    Zcur = tsp_cost(Xcur,costDict)
    
    Tcur = T0
    Xbest = Xcur
    Zbest = Zcur
    print('Current Solution : Xcur =',Xbest,'Zcur =',Zbest)
    
    #Simulated Annealing Procedure - Phase II
    for i in range(1,I+1):
        #Generate a neighbor solution, Xcount
        #call tsp_neighbor(route)
        Xcount=tsp_neighbor(X0)
        Z = tsp_cost(Xcount,costDict)
        print('i=',i)
        print('Xcount=',Xcount,'Zcount=',Z)

        if (Z<Zcur):
            print('Z-Zcur=',Z-Zcur)
            Xcur = Xcount
            Zcur = Z
            print('Xcur=',Xcur,'Zcur=',Z)


        else:
            C=Z-Zcur
            print('C=',C)
            if(random()<=m.exp(-C/Tcur)):
                Xcur = Xcount
                Zcur = Z
                print('Xcur=',Xcur,'Zcur=',Zcur)


        if(Z < Zbest):
            print('Z-Zbest=',Z-Zbest)
            Zbest = Z
            Xbest = Xcur
            print('Xbest=',Xbest,'Zbest=',Zbest)

        Tcur=Tcur-d
        print('Tcur=',Tcur)
        print('Xbest=',Xbest,'Zbest=',Zbest)
        
        runTime = time.time()-start_time
        print(runTime)
        
        #Simulated Annealing Procedure - Phase III
        if((Tcur<Tfinal) or(runTime>=cutoffTime)):
            break
            
    if(Tcur<Tfinal):
        print('Tcur=',Tcur,'< Tfinal=',Tfinal)
        print('runTime =',runTime,',cutoffTime =',cutoffTime)
    elif(runTime>=cutoffTime):
        print('runTime =',runTime,'> cutoffTime =',cutoffTime)
        print('Tcur=',Tcur,',Tfinal=',Tfinal)
    
    #totalruntime = time.time()-start_time
    #print(totalruntime)
    
    # SA produces a "best" solution, X_best.
    # X_best is just a sequence of node numbers,
    # starting from and returning to a "home" location.
    # For example, Xbest = [1, 5, 3, 4, 2, 1].
    
    # Your function should return a VeRoViz "assignments" dataframe.
    # Fortunately, VeRoViz provides a function to make this very easy:
    assignmentsDF = vrv.createAssignmentsFromNodeSeq2D(
        nodeSeq          = Xbest,
        nodes            = nodesDF,
        objectID         = 'Blue Car',
        modelFile        = 'veroviz/models/car_blue.gltf',
        modelScale       = 100,
        modelMinPxSize   = 75,
        routeType        = 'fastest',
        speedMPS         = None,
        leafletColor     = 'blue',
        leafletWeight    = 3,
        leafletStyle     = 'dashed',
        leafletOpacity   = 0.8,
        useArrows        = True,
        cesiumColor      = 'blue',
        cesiumWeight     = 3,
        cesiumStyle      = 'solid',
        cesiumOpacity    = 0.8,
        dataProvider     = 'ORS-online',
        dataProviderArgs = {'APIkey' : ORS_API_KEY})

    # See https://veroviz.org/docs/veroviz.createAssignments.html#veroviz.createAssignments.createAssignmentsFromNodeSeq2D
    # for more info on this function.
    
    return assignmentsDF
    #return Xbest

In [19]:
#%time
X_best=solveTSP_SA_loop(nodesDF,timeSec,3)
X_best

Intial Solution : X0 = [1, 9, 5, 2, 4, 7, 8, 6, 10, 3, 1] Z0 = 7060.207771879965
Current Solution : Xcur = [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcur = 7493.989289039374
i= 1
Xcount= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zcount= 7072.416861432164
Z-Zcur= -421.57242760720965
Xcur= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zcur= 7072.416861432164
Z-Zbest= -421.57242760720965
Xbest= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zbest= 7072.416861432164
Tcur= 3995
Xbest= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zbest= 7072.416861432164
0.012925148010253906
i= 2
Xcount= [1, 9, 5, 2, 4, 6, 8, 7, 10, 3, 1] Zcount= 8202.668961216606
C= 1130.2520997844422
Xcur= [1, 9, 5, 2, 4, 6, 8, 7, 10, 3, 1] Zcur= 8202.668961216606
Tcur= 3990
Xbest= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zbest= 7072.416861432164
0.01332402229309082
i= 3
Xcount= [1, 9, 5, 2, 4, 7, 8, 10, 6, 3, 1] Zcount= 7378.338627430442
Z-Zcur= -824.3303337861644
Xcur= [1, 9, 5, 2, 4, 7, 8, 10, 6, 3, 1] Zcur= 7378.338627430442
Tcur= 3985
Xbest= [1, 9, 5, 2, 4, 10, 6, 8, 7,

Xcount= [5, 9, 1, 2, 4, 7, 8, 6, 10, 3, 5] Zcount= 7692.172772419486
C= 0.0
Xcur= [5, 9, 1, 2, 4, 7, 8, 6, 10, 3, 5] Zcur= 7692.172772419486
Tcur= 3755
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
0.34870219230651855
i= 50
Xcount= [1, 9, 5, 2, 4, 6, 8, 7, 10, 3, 1] Zcount= 8202.668961216606
C= 510.49618879712034
Xcur= [1, 9, 5, 2, 4, 6, 8, 7, 10, 3, 1] Zcur= 8202.668961216606
Tcur= 3750
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
0.3489110469818115
i= 51
Xcount= [1, 9, 5, 3, 10, 6, 8, 7, 4, 2, 1] Zcount= 7692.172772419486
Z-Zcur= -510.49618879712034
Xcur= [1, 9, 5, 3, 10, 6, 8, 7, 4, 2, 1] Zcur= 7692.172772419486
Tcur= 3745
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
0.34909725189208984
i= 52
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
C= 446.9840441478409
Tcur= 3740
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
0.34935498237609863
i= 53
Xcount= [1, 4, 2, 5, 9, 7, 8, 6, 10,

Tcur= 3355
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.069533109664917
i= 130
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
C= 856.5453080945399
Xcur= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcur= 8139.156816567327
Tcur= 3350
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.0697441101074219
i= 131
Xcount= [1, 9, 5, 10, 6, 8, 7, 4, 2, 3, 1] Zcount= 7098.230941787261
Z-Zcur= -1040.925874780066
Xcur= [1, 9, 5, 10, 6, 8, 7, 4, 2, 3, 1] Zcur= 7098.230941787261
Tcur= 3345
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.0699310302734375
i= 132
Xcount= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zcount= 7072.416861432164
Z-Zcur= -25.81408035509685
Xcur= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zcur= 7072.416861432164
Tcur= 3340
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.0701580047607422
i= 133
Xcount= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcount= 7493.989289039374
C= 421.57242760720965
Xcur= [1, 9, 5, 2

Xcount= [1, 2, 5, 9, 4, 7, 8, 6, 10, 3, 1] Zcount= 7337.322584359886
C= 239.09164257262455
Xcur= [1, 2, 5, 9, 4, 7, 8, 6, 10, 3, 1] Zcur= 7337.322584359886
Tcur= 3085
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.437171220779419
i= 184
Xcount= [1, 9, 8, 7, 4, 2, 5, 6, 10, 3, 1] Zcount= 8391.497740920406
C= 1054.17515656052
Xcur= [1, 9, 8, 7, 4, 2, 5, 6, 10, 3, 1] Zcur= 8391.497740920406
Tcur= 3080
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.4383831024169922
i= 185
Xcount= [1, 7, 4, 2, 5, 9, 8, 6, 10, 3, 1] Zcount= 8574.615118704238
C= 183.11737778383213
Xcur= [1, 7, 4, 2, 5, 9, 8, 6, 10, 3, 1] Zcur= 8574.615118704238
Tcur= 3075
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.4391212463378906
i= 186
Xcount= [1, 9, 5, 10, 6, 8, 7, 4, 2, 3, 1] Zcount= 7098.230941787261
Z-Zcur= -1476.3841769169767
Xcur= [1, 9, 5, 10, 6, 8, 7, 4, 2, 3, 1] Zcur= 7098.230941787261
Tcur= 3070
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6

C= 1630.3864041081533
Tcur= 2815
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.650395154953003
i= 238
Xcount= [1, 9, 5, 2, 4, 7, 8, 6, 3, 10, 1] Zcount= 8139.156816567327
C= 1834.5356954961262
Tcur= 2810
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.6507151126861572
i= 239
Xcount= [1, 9, 5, 2, 3, 10, 6, 8, 7, 4, 1] Zcount= 6572.498543266443
C= 267.8774221952426
Xcur= [1, 9, 5, 2, 3, 10, 6, 8, 7, 4, 1] Zcur= 6572.498543266443
Tcur= 2805
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.6517019271850586
i= 240
Xcount= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcount= 7493.989289039374
C= 921.4907457729305
Xcur= [1, 9, 5, 2, 4, 7, 8, 3, 10, 6, 1] Zcur= 7493.989289039374
Tcur= 2800
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.652029275894165
i= 241
Xcount= [1, 9, 5, 4, 2, 7, 8, 6, 10, 3, 1] Zcount= 8631.66355278906
C= 1137.6742637496855
Tcur= 2795
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.62112107

Xcount= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zcount= 7072.416861432164
Z-Zcur= -1130.2520997844422
Xcur= [1, 9, 5, 2, 4, 10, 6, 8, 7, 3, 1] Zcur= 7072.416861432164
Tcur= 2405
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.9362690448760986
i= 320
Xcount= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zcount= 6304.621121071201
Z-Zcur= -767.7957403609635
Xcur= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zcur= 6304.621121071201
Tcur= 2400
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.9366090297698975
i= 321
Xcount= [1, 9, 5, 2, 4, 7, 8, 10, 6, 3, 1] Zcount= 7378.338627430442
C= 1073.7175063592413
Xcur= [1, 9, 5, 2, 4, 7, 8, 10, 6, 3, 1] Zcur= 7378.338627430442
Tcur= 2395
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] Zbest= 6304.621121071201
1.9374232292175293
i= 322
Xcount= [1, 9, 8, 7, 4, 2, 5, 6, 10, 3, 1] Zcount= 8391.497740920406
C= 1013.1591134899636
Xcur= [1, 9, 8, 7, 4, 2, 5, 6, 10, 3, 1] Zcur= 8391.497740920406
Tcur= 2390
Xbest= [1, 9, 5, 2, 10, 6, 8, 7, 4, 3, 1] 

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,ganttColor,popupText,startElevMeters,endElevMeters,wayname,waycategory,surface,waytype,steepness,tollway
0,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,0.000000,42.963622,-78.746143,0,18.400000,...,darkgray,None,204.0,204.0,Rock Street,No category,Asphalt,Street,0,False
1,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,18.400000,42.964298,-78.746319,0,22.290428,...,darkgray,None,204.0,204.0,Glen Avenue,No category,Asphalt,Street,0,False
2,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,22.290428,42.964244,-78.746504,0,33.832206,...,darkgray,None,204.0,204.0,Glen Avenue,No category,Asphalt,Street,0,False
3,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,33.832206,42.964136,-78.747076,0,37.300000,...,darkgray,None,204.0,204.0,Glen Avenue,No category,Asphalt,Street,0,False
4,1,Blue Car,/veroviz/models/car_blue.gltf,100,75,37.300000,42.964107,-78.747249,0,48.738125,...,darkgray,None,204.0,204.0,North Cayuga Road,No category,Asphalt,Street,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,7047.950922,42.962559,-78.746703,0,7052.430128,...,darkgray,None,204.6,204.3,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1779,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,7052.430128,42.962669,-78.746010,0,7053.400000,...,darkgray,None,204.3,204.2,"Main Street, NY 5",No category,Asphalt,State Road,0,False
1780,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,7053.400000,42.962693,-78.745860,0,7056.795360,...,darkgray,None,204.2,204.1,Rock Street,No category,Asphalt,Street,0,False
1781,10,Blue Car,/veroviz/models/car_blue.gltf,100,75,7056.795360,42.962817,-78.745899,0,7075.242069,...,darkgray,None,204.1,203.6,Rock Street,No category,Asphalt,Street,0,False


In [20]:
vrv.createLeaflet(arcs=X_best, nodes=nodesDF)

## Conclusion:

# Loop runs faster than recursive function